<a href="https://colab.research.google.com/github/Deeks900/GenAIBootcamp/blob/main/2_Text_Processing_Logistic_Regression_and_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression and Boosting Algorithms

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


## Predicting a Single Categorical Response
---



### Installing stuff

In [6]:
!pip install nbstripout


In [7]:
!pip install --upgrade textblob spacy gensim swifter 'numpy<1.26'

In [ ]:
!python -m textblob.download_corpora lite
!python -m spacy download en_core_web_sm


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
Finished.


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB         # Naive Bayes
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
import torch
import spacy
import gensim
import warnings
import nltk
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words


In [ ]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/scl/fi/dr6xmgw59kliq74gcd340/yelp.csv?rlkey=la6ue9a899v54f04eu92lbmlx&st=kjg7pmvm&dl=0
fi

In [ ]:
!bash get_data.sh

In [ ]:
# Read yelp.csv into a DataFrame.
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[ (yelp.stars == 1) | (yelp.stars == 5) ]

# Define X and y.
X = yelp_best_worst.text
y = yelp_best_worst.stars

# Split the new DataFrame into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

<a id="using-logistic-regression-for-classification"></a>
## Using Logistic Regression for Classification
---



In [ ]:
# Fit a logistic regression model to predict stars from text

logreg = LogisticRegression()

logreg.fit(X,y)


Of course this simply fails, we need to preprocess the text, convert it into a Tensor format and then and only then we can use models!

### Converting text to vectors

In [ ]:
import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    # text_token_list = [word for word in text.split(' ')
    #                         if word not in my_stopwords] # remove stopwords
    # text_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in text_token_list] # apply word rooter
    # text = ' '.join(text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [ ]:
preprocess_text("This is the best day of my life. Isn't it? Я тобі сама ", should_join=False)

In [ ]:
# Apply the preprocessing to the dataset
import swifter
X_preprocessed = X.swifter.apply(preprocess_text)

In [ ]:
X_preprocessed[0]

How do we pass from text to numbers? With tokenizers.

In [ ]:
# Implement this method
def get_maximum_review_length(srs):
    maximum = 0
    for review in srs:
      if len(review.split()) > maximum:
        maximum = len(review.split())
    return maximum


maximum = get_maximum_review_length(X_preprocessed)

In [ ]:
print(f'The maximum review was {maximum} words long')

In [ ]:
!pip install pytorch-nlp

In [ ]:
import itertools
from torchnlp.encoders import LabelEncoder

# Use the LabelEncoder to use as Tokenizer that transforms words into indices.
# Hint: You may need to create first a list of all words in your corpus

# WAY 1 : Double loop
# words = set()
# for list_of_words in X_preprocessed.apply(lambda x: x.split()):
#   for word in list_of_words:
#     words.add(word)
# ids_from_words = LabelEncoder(list(words))

ids_from_words = LabelEncoder(list(itertools.chain.from_iterable(X_preprocessed.swifter.apply(lambda x: x.split()))), reserved_labels=['[UNK]'], unknown_index=0)



In [ ]:
ids_from_words.vocab_size

In [ ]:
ids_from_words.batch_encode(["breakfast"])

In [ ]:
ids_from_words.decode(ids_from_words.encode("breakfast"))


In [ ]:
def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)

In [ ]:
def ids_from_text(text):
  return ids_from_words.batch_encode(text)

In [ ]:
ids = ids_from_text('Only you can prevent forest fires'.lower().split())
ids

In [ ]:
text_from_ids(ids)




In [ ]:
def pad_sequence_of_tokens(x, maxlen, unk_token='[UNK]'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [ ]:
def get_tensor(x, maximum=maximum):
  padding = (0, maximum-ids_from_text(x).shape[-1])
  return torch.squeeze(F.pad(ids_from_text(x), padding, "constant", 0).to(torch.long))


In [ ]:
import torch.nn.functional as F
# Very useful method
def get_ids_tensor(srs):

  processed = srs.swifter.apply(lambda x: pad_sequence_of_tokens(preprocess_text(x, should_join=False), maxlen=maximum))
  result = processed.swifter.apply(get_tensor).to_list()
  return torch.stack(result)


In [ ]:
all_ids = get_ids_tensor(srs=X_preprocessed.reset_index(drop=True))
all_ids

In [ ]:
all_ids.shape

In [ ]:
# Split the all_ids into.a train a test sets
X_train, X_test, y_train, y_test = train_test_split(all_ids, y, random_state=1)

### Using Logistic Regression

In [ ]:

# Train a Logistic Regression on X_train and give the accuracy
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


## Using Boosting Algorithms and other things

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

## Multiclass Classification

Just check in the estimators, most support multiclass classification.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, multi_class='multinomial').fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

### **Homework**: Try to perform the stars classification with Logistic Regression but without filtering only for 5 and 1 stars.